In [1]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import files
from oauth2client.client import GoogleCredentials
import pandas as pd
import io
from scipy.sparse import csr_matrix
import numpy as np
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
import nltk
import re
import pprint
import string
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk import pos_tag
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.feature_selection import mutual_info_classif
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score
from collections import Counter
from string import punctuation
from nltk.util import ngrams
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
stop_words = set(stopwords.words('english'))

## Import n-grams and word lists
import the lists of the n-grams,words, and negation words that were selected in the feature selection process in the information gain notebook.

In [3]:
uploaded = files.upload()

Saving gram_list.csv to gram_list.csv


In [0]:
grams = pd.read_csv(io.StringIO(uploaded['gram_list.csv'].decode('utf-8')))

In [6]:
grams = pd.read_csv('gram_list.csv')
grams = grams['feat'].tolist()
grams = set(grams)
len(grams)

912

In [7]:
uploaded = files.upload()

Saving word_list.csv to word_list.csv


In [0]:
words = pd.read_csv(io.StringIO(uploaded['word_list.csv'].decode('utf-8')))

In [9]:
words = pd.read_csv('word_list.csv')
words = words['feat'].tolist()
words = set(words)
len(words)

568

In [10]:
uploaded = files.upload()

Saving negation_list.csv to negation_list.csv


In [0]:
negate = pd.read_csv(io.StringIO(uploaded['negation_list.csv'].decode('utf-8')))

# Yelp Final Pipeline
Repeat process used in the feature engineering phase to create all features for prediction but only use the chosen features in the lists above and the other features chosen in the information gain extraction notebook.

In [12]:
uploaded = files.upload()

Saving yelp2.csv to yelp2.csv


In [0]:
yelp = pd.read_csv(io.StringIO(uploaded['yelp2.csv'].decode('utf-8')))

In [0]:
yelp = yelp.rename(columns={'positive':'pos_class'})

In [0]:
def lemmatize_text(text):
    lemmatizer = nltk.stem.WordNetLemmatizer()
    words = nltk.word_tokenize(text)
    words=[word.lower() for word in words if word.isalpha()]
    return [lemmatizer.lemmatize(w, pos='v') for w in words]

#function to remove # and numbers from General Inquirer positive and negative word lists
def clean_list(mylist):
    import re
    lis = []
    for word in mylist:
        word = word.lower()
        word = re.sub('[#0-9]','', word)    
        lis.append(word)
    myset = set(lis)
    cleaned_list = list(myset)
    return cleaned_list

In [16]:
#clean \n out of text
clean_text = []
for t in tqdm(yelp['text']):
    temp = []
    t = t.replace('\n','')
    temp.append(t)
    clean_text.append(temp)
clean_text = pd.DataFrame(clean_text, columns=['clean_text'])
yelp = yelp.join(clean_text)

100%|██████████| 99579/99579 [00:00<00:00, 389320.87it/s]


In [17]:
#create lemmas feature
lemmas = [lemmatize_text(yelp.text[x]) for x in tqdm(range(len(yelp['text']))) ]

100%|██████████| 99579/99579 [02:50<00:00, 583.97it/s]


In [0]:
yelp['lemmas'] = lemmas

In [19]:
#create binary feature if negations are present
negations = []
for word in tqdm(yelp['lemmas']):
    count =0
    for w in word:
        if w.lower() in negate:
            count+=1
    if count >= 1:
        negations.append(1)
    else:
        negations.append(0)
            

100%|██████████| 99579/99579 [00:11<00:00, 8472.34it/s]


In [0]:
yelp['negate'] = negations

In [21]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()
for i,r in tqdm(yelp.iterrows(), total=yelp.shape[0]):
    text = r['text']
    counts = Counter(text)
    punctuation_counts = {k:v for k, v in counts.items() if k in punctuation}
    pc = sum(punctuation_counts.values())
    yelp.at[i, 'punc_count'] = pc
    
    text_l = text.lower()
    word_tokens = word_tokenize(text_l)
    stop_w = [word for word in word_tokens if word in stop_words]
    percent_stop = round((len(stop_w)/len(word_tokens)), 2)
    yelp.at[i, 'percent_stop'] = percent_stop
    
    filtered_sentence = []
    for w in word_tokens: 
        if w not in stop_words: 
            filtered_sentence.append(w) 
    Stem_words = []
    for w in filtered_sentence:
        rootWord=ps.stem(w)
        Stem_words.append(rootWord)
    yelp.at[i, 'length_preprocessed_review'] = len(Stem_words)

100%|██████████| 99579/99579 [04:40<00:00, 355.60it/s]


In [0]:
#sentence count 
num_sent = []
for i in yelp['text']:
    num_sent.append(len(sent_tokenize(i)))
yelp['sent_count'] = num_sent

#word count without puncuation
tokenizer_words = RegexpTokenizer(r'\w+')
num_words = []
for i in yelp['text']:
    num_words.append(len(tokenizer_words.tokenize(i)))
yelp['word_count'] = num_words

#count exclamations in reviews
tokenizer_exclamation = RegexpTokenizer(r'\!')
num_exclaim = []
for i in yelp['text']:
    num_exclaim.append(len(tokenizer_exclamation.tokenize(i)))
yelp['exclaim_count'] = num_exclaim

#create average sentence length variable
yelp['avg_sent']= yelp['word_count']/yelp['sent_count']


In [0]:
yelp = yelp.drop(['length_preprocessed_review','sent_count'],axis=1)

In [0]:
y = yelp['pos_class']
X_train, X_test, y_train, y_test = train_test_split(yelp, y, test_size=0.3, random_state=5, shuffle=True)

In [0]:
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
X_train = X_train.reset_index(drop=True)

In [0]:
X_train_scaled = X_train[['punc_count','percent_stop','word_count','exclaim_count','avg_sent']]
X_test_scaled = X_test[['punc_count','percent_stop','word_count','exclaim_count','avg_sent']]
X_train = X_train.drop(['punc_count','percent_stop','word_count','exclaim_count','avg_sent'],axis=1)
X_test = X_test.drop(['punc_count','percent_stop','word_count','exclaim_count','avg_sent'],axis=1)

In [0]:
columns = ['punc_count','percent_stop','word_count','exclaim_count','avg_sent']

In [0]:
#scale non-vetorized features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_scaled)
X_test_scaled = scaler.transform(X_test_scaled)

In [0]:
X_train_scaled = pd.DataFrame(X_train_scaled,columns=columns )
X_test_scaled = pd.DataFrame(X_test_scaled,columns=columns )

In [0]:
X_train = X_train.join(X_train_scaled)
X_test = X_test.join(X_test_scaled)

In [0]:
#vectorize n-grams
gramvec = TfidfVectorizer(vocabulary=grams,analyzer='char', ngram_range=(2, 3))
X_train_count_grams = gramvec.fit_transform(X_train['clean_text'])


In [0]:
gram_labels = gramvec.get_feature_names()
gram_matrix = X_train_count_grams.todense()

In [0]:
gram_matrix = pd.DataFrame(gram_matrix, columns=gram_labels)
X_train = X_train.join(gram_matrix)

In [0]:
#test data
X_test_count_grams = gramvec.transform(X_test['clean_text'])

In [0]:
gram_labels = gramvec.get_feature_names()
gram_matrix = X_test_count_grams.todense()
gram_matrix = pd.DataFrame(gram_matrix, columns=gram_labels)
X_test = X_test.join(gram_matrix)

In [0]:
#vectorize words
wordvec = TfidfVectorizer(vocabulary=words)
X_train_count_words = wordvec.fit_transform(X_train['clean_text'])

word_labels = wordvec.get_feature_names()
word_matrix = X_train_count_words.todense()

word_matrix = pd.DataFrame(word_matrix, columns=word_labels)
X_train = X_train.join(word_matrix)

#test data
X_test_count_words = wordvec.transform(X_test['clean_text'])

word_labels = wordvec.get_feature_names()
word_matrix = X_test_count_words.todense()

word_matrix = pd.DataFrame(word_matrix, columns=word_labels)
X_test = X_test.join(word_matrix)

In [37]:
#drop unused columns and transform to dense matrix
X_train = X_train.drop(['text','pos_class','clean_text','lemmas'], axis=1)
X_test = X_test.drop(['text','pos_class','clean_text','lemmas'], axis=1)
columns = X_train.columns

X_train = X_train.to_numpy()
X_test = X_test.to_numpy()

X_train = csr_matrix(X_train)
X_test = csr_matrix(X_test)

sgd = SGDClassifier(verbose=1, random_state=5,n_jobs=-1)
sgd.fit(X_train,y_train)

-- Epoch 1
Norm: 33.71, NNZs: 1485, Bias: -1.090212, T: 69705, Avg. loss: 0.602272
Total training time: 0.07 seconds.
-- Epoch 2
Norm: 30.08, NNZs: 1485, Bias: -1.064068, T: 139410, Avg. loss: 0.245700
Total training time: 0.14 seconds.
-- Epoch 3
Norm: 28.78, NNZs: 1485, Bias: -1.054096, T: 209115, Avg. loss: 0.227840
Total training time: 0.22 seconds.
-- Epoch 4
Norm: 28.30, NNZs: 1485, Bias: -1.039098, T: 278820, Avg. loss: 0.220000
Total training time: 0.29 seconds.
-- Epoch 5
Norm: 27.93, NNZs: 1485, Bias: -1.019556, T: 348525, Avg. loss: 0.215046
Total training time: 0.36 seconds.
-- Epoch 6
Norm: 27.67, NNZs: 1485, Bias: -1.002907, T: 418230, Avg. loss: 0.212822
Total training time: 0.44 seconds.
-- Epoch 7
Norm: 27.56, NNZs: 1485, Bias: -0.989446, T: 487935, Avg. loss: 0.210528
Total training time: 0.51 seconds.
-- Epoch 8
Norm: 27.43, NNZs: 1485, Bias: -0.982277, T: 557640, Avg. loss: 0.209408
Total training time: 0.58 seconds.
-- Epoch 9
Norm: 27.34, NNZs: 1485, Bias: -0.9743

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=-1, penalty='l2',
              power_t=0.5, random_state=5, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=1, warm_start=False)

In [38]:
#accuracy of classification of positive and negative reviews
y_pred = sgd.predict(X_test)
print('accuracy %s' % accuracy_score(y_test,y_pred))

accuracy 0.9160139251523064


# IMDB Final Pipeline
repeat process above for the IMDB dataset

In [39]:
uploaded = files.upload()

Saving imdb_dataset.csv to imdb_dataset.csv


In [0]:
imdb = pd.read_csv(io.StringIO(uploaded['imdb_dataset.csv'].decode('utf-8')))

In [0]:
one_hot = pd.get_dummies(imdb['sentiment'])
imdb = imdb.join(one_hot)
imdb = imdb.drop(['sentiment','negative'], axis=1)

imdb = imdb.rename(columns={'review':'text','positive':'pos_class'})

In [42]:
clean_text = []
for t in tqdm(imdb['text']):
    temp = []
    t = t.replace('\n','')
    t = t.replace('<br','')
    t = t.replace('/>','')
    temp.append(t)
    clean_text.append(temp)
clean_text = pd.DataFrame(clean_text, columns=['clean_text'])
imdb = imdb.join(clean_text)

100%|██████████| 50000/50000 [00:00<00:00, 67899.47it/s]


In [43]:
#create lemmas feature
lemmas = [lemmatize_text(imdb.text[x]) for x in tqdm(range(len(imdb['text']))) ]

imdb['lemmas'] = lemmas

#create binary feature if negations are present
negations = []
for word in tqdm(imdb['lemmas']):
    count =0
    for w in word:
        if w.lower() in negate:
            count+=1
    if count >= 1:
        negations.append(1)
    else:
        negations.append(0)
            
imdb['negate'] = negations

100%|██████████| 50000/50000 [00:11<00:00, 4413.54it/s]


In [44]:
ps = PorterStemmer()
for i,r in tqdm(imdb.iterrows(), total=imdb.shape[0]):
    text = r['text']
    counts = Counter(text)
    punctuation_counts = {k:v for k, v in counts.items() if k in punctuation}
    pc = sum(punctuation_counts.values())
    imdb.at[i, 'punc_count'] = pc
    
    text_l = text.lower()
    word_tokens = word_tokenize(text_l)
    stop_w = [word for word in word_tokens if word in stop_words]
    percent_stop = round((len(stop_w)/len(word_tokens)), 2)
    imdb.at[i, 'percent_stop'] = percent_stop
    
    filtered_sentence = []
    for w in word_tokens: 
        if w not in stop_words: 
            filtered_sentence.append(w) 
    Stem_words = []
    for w in filtered_sentence:
        rootWord=ps.stem(w)
        Stem_words.append(rootWord)
    imdb.at[i, 'length_preprocessed_review'] = len(Stem_words)

#sentence count 
num_sent = []
for i in imdb['text']:
    num_sent.append(len(sent_tokenize(i)))
imdb['sent_count'] = num_sent

#word count without puncuation
tokenizer_words = RegexpTokenizer(r'\w+')
num_words = []
for i in imdb['text']:
    num_words.append(len(tokenizer_words.tokenize(i)))
imdb['word_count'] = num_words

#count exclamations in reviews
tokenizer_exclamation = RegexpTokenizer(r'\!')
num_exclaim = []
for i in imdb['text']:
    num_exclaim.append(len(tokenizer_exclamation.tokenize(i)))
imdb['exclaim_count'] = num_exclaim

#create average sentence length variable
imdb['avg_sent']= imdb['word_count']/imdb['sent_count']



imdb = imdb.drop(['length_preprocessed_review','sent_count'],axis=1)


y = imdb['pos_class']
X_train, X_test, y_train, y_test = train_test_split(imdb, y, test_size=0.3, random_state=5, shuffle=True)

y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
X_train = X_train.reset_index(drop=True)

X_train_scaled = X_train[['punc_count','percent_stop','word_count','exclaim_count','avg_sent']]
X_test_scaled = X_test[['punc_count','percent_stop','word_count','exclaim_count','avg_sent']]
X_train = X_train.drop(['punc_count','percent_stop','word_count','exclaim_count','avg_sent'],axis=1)
X_test = X_test.drop(['punc_count','percent_stop','word_count','exclaim_count','avg_sent'],axis=1)

columns = ['punc_count','percent_stop','word_count','exclaim_count','avg_sent']

  0%|          | 21/50000 [00:00<04:04, 204.40it/s]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


100%|██████████| 50000/50000 [03:55<00:00, 212.31it/s]


In [0]:
#scale non-vetorized features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_scaled)
X_test_scaled = scaler.transform(X_test_scaled)

X_train_scaled = pd.DataFrame(X_train_scaled,columns=columns )
X_test_scaled = pd.DataFrame(X_test_scaled,columns=columns )

X_train = X_train.join(X_train_scaled)
X_test = X_test.join(X_test_scaled)

In [0]:
#vectorize n-grams
gramvec = TfidfVectorizer(vocabulary=grams,analyzer='char', ngram_range=(2, 3))
X_train_count_grams = gramvec.fit_transform(X_train['clean_text'])


In [0]:
gram_labels = gramvec.get_feature_names()
gram_matrix = X_train_count_grams.todense()

In [0]:
gram_matrix = pd.DataFrame(gram_matrix, columns=gram_labels)
X_train = X_train.join(gram_matrix)

In [0]:
#test data
X_test_count_grams = gramvec.transform(X_test['clean_text'])

In [0]:
gram_labels = gramvec.get_feature_names()
gram_matrix = X_test_count_grams.todense()
gram_matrix = pd.DataFrame(gram_matrix, columns=gram_labels)
X_test = X_test.join(gram_matrix)

In [0]:
#vectorize words
wordvec = TfidfVectorizer(vocabulary=words)
X_train_count_words = wordvec.fit_transform(X_train['clean_text'])

word_labels = wordvec.get_feature_names()
word_matrix = X_train_count_words.todense()

word_matrix = pd.DataFrame(word_matrix, columns=word_labels)
X_train = X_train.join(word_matrix)

#test data
X_test_count_words = wordvec.transform(X_test['clean_text'])

word_labels = wordvec.get_feature_names()
word_matrix = X_test_count_words.todense()

word_matrix = pd.DataFrame(word_matrix, columns=word_labels)
X_test = X_test.join(word_matrix)

In [52]:
#drop unused columns and transform to dense matrix
X_train = X_train.drop(['text','pos_class','clean_text','lemmas'], axis=1)
X_test = X_test.drop(['text','pos_class','clean_text','lemmas'], axis=1)
columns = X_train.columns

X_train = X_train.to_numpy()
X_test = X_test.to_numpy()

X_train = csr_matrix(X_train)
X_test = csr_matrix(X_test)

sgd_imdb = SGDClassifier(verbose=1, random_state=5,n_jobs=-1)
sgd_imdb.fit(X_train,y_train)

-- Epoch 1
Norm: 40.06, NNZs: 1485, Bias: 0.314121, T: 35000, Avg. loss: 1.478375
Total training time: 0.03 seconds.
-- Epoch 2
Norm: 33.83, NNZs: 1485, Bias: 0.351292, T: 70000, Avg. loss: 0.531989
Total training time: 0.07 seconds.
-- Epoch 3
Norm: 32.15, NNZs: 1485, Bias: 0.387574, T: 105000, Avg. loss: 0.474561
Total training time: 0.10 seconds.
-- Epoch 4
Norm: 31.05, NNZs: 1485, Bias: 0.404210, T: 140000, Avg. loss: 0.441306
Total training time: 0.14 seconds.
-- Epoch 5
Norm: 30.46, NNZs: 1485, Bias: 0.418651, T: 175000, Avg. loss: 0.422657
Total training time: 0.17 seconds.
-- Epoch 6
Norm: 30.10, NNZs: 1485, Bias: 0.434421, T: 210000, Avg. loss: 0.414743
Total training time: 0.21 seconds.
-- Epoch 7
Norm: 29.74, NNZs: 1485, Bias: 0.443621, T: 245000, Avg. loss: 0.401328
Total training time: 0.24 seconds.
-- Epoch 8
Norm: 29.63, NNZs: 1485, Bias: 0.452056, T: 280000, Avg. loss: 0.397449
Total training time: 0.28 seconds.
-- Epoch 9
Norm: 29.46, NNZs: 1485, Bias: 0.457004, T: 315

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=-1, penalty='l2',
              power_t=0.5, random_state=5, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=1, warm_start=False)

In [53]:
#accuracy of classification of positive and negative reviews
y_pred = sgd_imdb.predict(X_test)
print('accuracy %s' % accuracy_score(y_test,y_pred))

accuracy 0.8408


# Coursera Test
Repeat process above on dataset not used in selecting the features.

In [55]:
uploaded = files.upload()

Saving coursera_clean.csv to coursera_clean.csv


In [0]:
coursera = pd.read_csv(io.StringIO(uploaded['coursera_clean.csv'].decode('utf-8')))

In [0]:
coursera = coursera.reset_index(drop=True)
coursera = coursera.rename(columns={'Review':'text','positive':'pos_class'})

In [60]:
coursera.head()

,text,pos_class
0,I thoroughly enjoyed it. The course material c...,1
1,I thoroughly enjoyed doing this course. I have...,1
2,Great and intuitive course.,1
3,Thoroughly enjoyed this course! Problems and a...,1
4,"Than; you very very much, i hope i can join yo...",1


In [61]:
clean_text = []
for t in tqdm(coursera['text']):
    temp = []
    t = t.replace('\n','')
    t = t.replace('<br','')
    t = t.replace('/>','')
    temp.append(t)
    clean_text.append(temp)
clean_text = pd.DataFrame(clean_text, columns=['clean_text'])
coursera = coursera.join(clean_text)

100%|██████████| 8000/8000 [00:00<00:00, 413619.05it/s]


In [62]:
#create lemmas feature
lemmas = [lemmatize_text(coursera.text[x]) for x in tqdm(range(len(coursera['text']))) ]

coursera['lemmas'] = lemmas

#create binary feature if negations are present
negations = []
for word in tqdm(coursera['lemmas']):
    count =0
    for w in word:
        if w.lower() in negate:
            count+=1
    if count >= 1:
        negations.append(1)
    else:
        negations.append(0)
            
coursera['negate'] = negations

100%|██████████| 8000/8000 [00:00<00:00, 24597.39it/s]


In [63]:
ps = PorterStemmer()
for i,r in tqdm(coursera.iterrows(), total=coursera.shape[0]):
    text = r['text']
    counts = Counter(text)
    punctuation_counts = {k:v for k, v in counts.items() if k in punctuation}
    pc = sum(punctuation_counts.values())
    coursera.at[i, 'punc_count'] = pc
    
    text_l = text.lower()
    word_tokens = word_tokenize(text_l)
    stop_w = [word for word in word_tokens if word in stop_words]
    percent_stop = round((len(stop_w)/len(word_tokens)), 2)
    coursera.at[i, 'percent_stop'] = percent_stop
    
    filtered_sentence = []
    for w in word_tokens: 
        if w not in stop_words: 
            filtered_sentence.append(w) 
    Stem_words = []
    for w in filtered_sentence:
        rootWord=ps.stem(w)
        Stem_words.append(rootWord)
    coursera.at[i, 'length_preprocessed_review'] = len(Stem_words)

  1%|▏         | 100/8000 [00:00<00:07, 995.94it/s]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


100%|██████████| 8000/8000 [00:08<00:00, 919.25it/s]


In [0]:
#sentence count 
num_sent = []
for i in coursera['text']:
    num_sent.append(len(sent_tokenize(i)))
coursera['sent_count'] = num_sent

#word count without puncuation
tokenizer_words = RegexpTokenizer(r'\w+')
num_words = []
for i in coursera['text']:
    num_words.append(len(tokenizer_words.tokenize(i)))
coursera['word_count'] = num_words

#count exclamations in reviews
tokenizer_exclamation = RegexpTokenizer(r'\!')
num_exclaim = []
for i in coursera['text']:
    num_exclaim.append(len(tokenizer_exclamation.tokenize(i)))
coursera['exclaim_count'] = num_exclaim

#create average sentence length variable
coursera['avg_sent']= coursera['word_count']/coursera['sent_count']

coursera = coursera.drop(['length_preprocessed_review','sent_count'],axis=1)

In [65]:
coursera

,text,pos_class,clean_text,lemmas,negate,punc_count,percent_stop,word_count,exclaim_count,avg_sent
0,I thoroughly enjoyed it. The course material c...,1,I thoroughly enjoyed it. The course material c...,"[i, thoroughly, enjoy, it, the, course, materi...",0,5.0,0.40,47,0,15.666667
1,I thoroughly enjoyed doing this course. I have...,1,I thoroughly enjoyed doing this course. I have...,"[i, thoroughly, enjoy, do, this, course, i, ha...",0,2.0,0.56,25,0,12.500000
2,Great and intuitive course.,1,Great and intuitive course.,"[great, and, intuitive, course]",0,1.0,0.20,4,0,4.000000
3,Thoroughly enjoyed this course! Problems and a...,1,Thoroughly enjoyed this course! Problems and a...,"[thoroughly, enjoy, this, course, problems, an...",0,1.0,0.31,12,1,6.000000
4,"Than; you very very much, i hope i can join yo...",1,"Than; you very very much, i hope i can join yo...","[than, you, very, very, much, i, hope, i, can,...",0,2.0,0.57,12,0,12.000000
...,...,...,...,...,...,...,...,...,...,...
7995,"Interresting, but too easy to understand. Not ...",0,"Interresting, but too easy to understand. Not ...","[interresting, but, too, easy, to, understand,...",0,6.0,0.33,19,0,6.333333
7996,Very simplistic concepts and tactics which are...,0,Very simplistic concepts and tactics which are...,"[very, simplistic, concepts, and, tactics, whi...",0,2.0,0.59,32,0,16.000000
7997,"This is the worst, most pathetic course I have...",0,"This is the worst, most pathetic course I have...","[this, be, the, worst, most, pathetic, course,...",0,24.0,0.49,148,0,13.454545
7998,Videos are very brief - slide presentations of...,0,Videos are very brief - slide presentations of...,"[videos, be, very, brief, slide, presentations...",0,3.0,0.33,21,0,10.500000


In [66]:
y = coursera['pos_class']
X_train, X_test, y_train, y_test = train_test_split(coursera, y, test_size=0.3, random_state=5, shuffle=True)

y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
X_train = X_train.reset_index(drop=True)

X_train_scaled = X_train[['punc_count','percent_stop','word_count','exclaim_count','avg_sent']]
X_test_scaled = X_test[['punc_count','percent_stop','word_count','exclaim_count','avg_sent']]
X_train = X_train.drop(['punc_count','percent_stop','word_count','exclaim_count','avg_sent'],axis=1)
X_test = X_test.drop(['punc_count','percent_stop','word_count','exclaim_count','avg_sent'],axis=1)

columns = ['punc_count','percent_stop','word_count','exclaim_count','avg_sent']

#scale non-vectorized features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_scaled)
X_test_scaled = scaler.transform(X_test_scaled)

X_train_scaled = pd.DataFrame(X_train_scaled,columns=columns )
X_test_scaled = pd.DataFrame(X_test_scaled,columns=columns )

X_train = X_train.join(X_train_scaled)
X_test = X_test.join(X_test_scaled)

#vectorize n-grams
gramvec = TfidfVectorizer(vocabulary=grams,analyzer='char', ngram_range=(2, 3))
X_train_count_grams = gramvec.fit_transform(X_train['clean_text'])


gram_labels = gramvec.get_feature_names()
gram_matrix = X_train_count_grams.todense()

gram_matrix = pd.DataFrame(gram_matrix, columns=gram_labels)
X_train = X_train.join(gram_matrix)

#test data
X_test_count_grams = gramvec.transform(X_test['clean_text'])

gram_labels = gramvec.get_feature_names()
gram_matrix = X_test_count_grams.todense()
gram_matrix = pd.DataFrame(gram_matrix, columns=gram_labels)
X_test = X_test.join(gram_matrix)

#vectorize words
wordvec = TfidfVectorizer(vocabulary=words)
X_train_count_words = wordvec.fit_transform(X_train['clean_text'])

word_labels = wordvec.get_feature_names()
word_matrix = X_train_count_words.todense()

word_matrix = pd.DataFrame(word_matrix, columns=word_labels)
X_train = X_train.join(word_matrix)

#test data
X_test_count_words = wordvec.transform(X_test['clean_text'])

word_labels = wordvec.get_feature_names()
word_matrix = X_test_count_words.todense()

word_matrix = pd.DataFrame(word_matrix, columns=word_labels)
X_test = X_test.join(word_matrix)

#drop unused columns and transform to dense matrix
X_train = X_train.drop(['text','pos_class','clean_text','lemmas'], axis=1)
X_test = X_test.drop(['text','pos_class','clean_text','lemmas'], axis=1)
columns = X_train.columns

X_train = X_train.to_numpy()
X_test = X_test.to_numpy()

X_train = csr_matrix(X_train)
X_test = csr_matrix(X_test)

sgd_coursera = SGDClassifier(verbose=1, random_state=5,n_jobs=-1)
sgd_coursera.fit(X_train,y_train)

-- Epoch 1
Norm: 76.89, NNZs: 1243, Bias: -1.778057, T: 5600, Avg. loss: 3.160814
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 58.96, NNZs: 1292, Bias: -1.977654, T: 11200, Avg. loss: 0.850673
Total training time: 0.00 seconds.
-- Epoch 3
Norm: 50.43, NNZs: 1303, Bias: -1.927956, T: 16800, Avg. loss: 0.512798
Total training time: 0.01 seconds.
-- Epoch 4
Norm: 46.58, NNZs: 1312, Bias: -1.857794, T: 22400, Avg. loss: 0.432507
Total training time: 0.01 seconds.
-- Epoch 5
Norm: 43.04, NNZs: 1316, Bias: -1.826849, T: 28000, Avg. loss: 0.341546
Total training time: 0.01 seconds.
-- Epoch 6
Norm: 41.31, NNZs: 1321, Bias: -1.795540, T: 33600, Avg. loss: 0.318132
Total training time: 0.01 seconds.
-- Epoch 7
Norm: 39.79, NNZs: 1329, Bias: -1.775582, T: 39200, Avg. loss: 0.306621
Total training time: 0.01 seconds.
-- Epoch 8
Norm: 38.80, NNZs: 1335, Bias: -1.751837, T: 44800, Avg. loss: 0.285376
Total training time: 0.01 seconds.
-- Epoch 9
Norm: 37.80, NNZs: 1336, Bias: -1.714606, T: 5

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=-1, penalty='l2',
              power_t=0.5, random_state=5, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=1, warm_start=False)

In [67]:
#accuracy of classification of positive and negative reviews
y_pred = sgd_coursera.predict(X_test)
print('accuracy %s' % accuracy_score(y_test,y_pred))

accuracy 0.8966666666666666
